In [1]:

import tensorflow as tf
from keras.src.layers import Bidirectional
from tensorflow.keras import mixed_precision
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
mixed_precision.set_global_policy('mixed_float16')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split

dataset = pd.read_csv('data/imdb_dataset.csv')


2025-12-13 01:35:52.328956: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-13 01:35:52.810397: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-13 01:36:02.546256: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


Num GPUs Available:  1


In [2]:
import numpy as np

#Define GloVe parameters
GLOVE_FILE = 'data/glove.6B.100d.txt'
EMBEDDING_DIM = 100

#Load GloVe vectors into a dictionary
embeddings_index = {}
print(f"Loading GloVe vectors from {GLOVE_FILE}...")

try:
    with open(GLOVE_FILE, encoding='utf8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = vector
except FileNotFoundError:
    print(f"\n*** ERROR: GloVe file not found. ***")
    print(f"Please download '{GLOVE_FILE}' and place it in your project folder.")

print(f"Successfully loaded {len(embeddings_index)} word vectors.")

Loading GloVe vectors from data/glove.6B.100d.txt...
Successfully loaded 400000 word vectors.


Split the original DataFrame into train and test sets
80% for training, 20% for testing.

In [3]:
train_df, test_df = train_test_split(dataset, test_size=0.2, random_state=42)

print(f"Full dataset size: {len(dataset)}")
print(f"Training set size: {len(train_df)}")
print(f"Test set size: {len(test_df)}")
train_df.head(10)

Full dataset size: 50000
Training set size: 40000
Test set size: 10000


,review,sentiment
39087,That's what I kept asking myself during the ma...,negative
30893,I did not watch the entire movie. I could not ...,negative
45278,A touching love story reminiscent of In the M...,positive
16398,This latter-day Fulci schlocker is a totally a...,negative
13653,"First of all, I firmly believe that Norwegian ...",negative
13748,I don't know how this movie received so many p...,negative
23965,Nightmare Weekend stars a cast of ridiculous a...,negative
45552,":::SPOILER ALERT:::<br /><br />Soooo, Arnie's ...",negative
30219,The people who are bad-mouthing this film are ...,positive
24079,"<br /><br />As usual, I was really looking for...",negative


Defining dataset parameters

In [4]:
VOCAB_SIZE = 8000
MAX_LEN = 250

Tokenization

In [5]:
# Create the 0/1 labels
y_train = train_df['sentiment'].apply(lambda x: 1 if x == 'positive' else 0).values
y_test = test_df['sentiment'].apply(lambda x: 1 if x == 'positive' else 0).values

# Process Reviews (X data)
tokenizer = Tokenizer(num_words=VOCAB_SIZE)

#Fit the tokenizer ONLY on the training text
tokenizer.fit_on_texts(train_df['review'])

# transform both train and test text
X_train = tokenizer.texts_to_sequences(train_df['review'])
X_test = tokenizer.texts_to_sequences(test_df['review'])

#Pad the sequences
X_train = pad_sequences(X_train, maxlen=MAX_LEN)
X_test = pad_sequences(X_test, maxlen=MAX_LEN)

print("\n--- First 10 rows of processed X_train (numbers) ---")
print(X_train[:10])
np.set_printoptions(threshold=np.inf)
print("\n--- Full 1st row of processed X_train ---")
print(X_train[0])
np.set_printoptions(threshold=0)


--- First 10 rows of processed X_train (numbers) ---
[[  13   92  485 ...  205  351 3856]
 [ 438   17   10 ...   89  103    9]
 [   0    0    0 ...    2  710   62]
 ...
 [  47 1025    8 ...  661  335  155]
 [   0    0    0 ...   17   63    6]
 [   5  131   71 ...  483    7    7]]

--- Full 1st row of processed X_train ---
[  13   92  485 6830   15    3  364 1182   61    8    1  470  216 1014
    5 4160    8    3  174    4   34  440  697  623   12 3748  237  111
  848   35  170   30  219  197    1  428  367   55 3765    3  278    7
    7  157 1707  187    6    1  727 1935    1 1200    4 2946 3749 1828
    2  147  144    3  228    4    3  207  323    2  144 1083   16   88
    4  132 2871   18   10  153   99    4    1 4020  302   11   17 1001
   35    1  496  492 2619  249   71   77  107  107  698   60   86 1047
 1363    5  229  132   23 4360   31  138  209 1154   14 4501 5313   31
    3 2386    2    8   11    6    3  445   14  624    4    1  718 2959
    1 1278    2   71 3616    1  166 

In [6]:
# Get your tokenizer's vocabulary
VOCAB_SIZE_FINAL = tokenizer.num_words + 1
embedding_matrix = np.zeros((VOCAB_SIZE_FINAL, EMBEDDING_DIM))
words_found = 0
print(f"Building embedding matrix for top {VOCAB_SIZE} words...")
for word, i in tokenizer.word_index.items():
    if i >= VOCAB_SIZE_FINAL:
        continue

    embedding_vector = embeddings_index.get(word)

    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        words_found += 1

print(f"Done. Found {words_found} / {VOCAB_SIZE} words in GloVe.")

Building embedding matrix for top 8000 words...
Done. Found 7858 / 8000 words in GloVe.


## Keras Tuner Setup for Automatic Hyperparameter Tuning

In [7]:
# Install Keras Tuner (run once)
# !pip install keras-tuner

import keras_tuner as kt
from keras_tuner import BayesianOptimization
from keras.src.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

print("Keras Tuner imported")

Keras Tuner imported


## Tunable Model Builder Function

In [8]:
def build_tunable_model(hp):
    num_layers = hp.Int("num_lstm_layers", min_value=1, max_value=3, default=2)
    lstm_units = hp.Choice("lstm_units", values=[50, 64, 100, 128, 192, 256])
    dropout = hp.Float("dropout", min_value=0.0, max_value=0.5, step=0.05, default=0.12)
    learning_rate = hp.Float("learning_rate", min_value=1e-5, max_value=1e-3, sampling="log", default=5e-5)
    
    model = Sequential()
    model.add(Embedding(input_dim=VOCAB_SIZE_FINAL, output_dim=EMBEDDING_DIM, weights=[embedding_matrix], trainable=True, input_length=MAX_LEN))
    model.add(Bidirectional(LSTM(units=lstm_units, dropout=dropout, return_sequences=(num_layers > 1))))
    
    for i in range(1, num_layers):
        return_seq = (i < num_layers - 1)
        model.add(LSTM(units=lstm_units, dropout=dropout, return_sequences=return_seq))
    
    model.add(Dense(units=1, activation="sigmoid"))
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss="binary_crossentropy", metrics=["accuracy"])
    return model

print("Model builder ready")

Model builder ready


## Configure Bayesian Optimization Tuner

In [9]:
tuner = BayesianOptimization(
    hypermodel=build_tunable_model,
    objective="val_accuracy",
    max_trials=75,
    executions_per_trial=1,
    directory="tuner_results",
    project_name="lstm_sentiment_tuning",
    overwrite=False,
    seed=42
)

print("Tuner configured: 75 trials, Bayesian Optimization")

Reloading Tuner from tuner_results/lstm_sentiment_tuning/tuner0.json
Tuner configured: 75 trials, Bayesian Optimization


I0000 00:00:1765582592.718079   24466 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5518 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4070 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


## Early Stopping Callbacks

In [10]:
early_stopping = EarlyStopping(monitor="val_accuracy", patience=3, restore_best_weights=True, mode="max", verbose=1)

class TrialPruning(tf.keras.callbacks.Callback):
    def __init__(self, min_accuracy=0.70):
        super().__init__()
        self.min_accuracy = min_accuracy
    def on_epoch_end(self, epoch, logs=None):
        if epoch >= 1 and logs.get("val_accuracy", 0) < self.min_accuracy:
            print(f"Stopping trial: val_accuracy {logs['val_accuracy']:.4f} < {self.min_accuracy}")
            self.model.stop_training = True

trial_pruning = TrialPruning(min_accuracy=0.70)
tuner_callbacks = [early_stopping, trial_pruning]
print("Callbacks configured")

Callbacks configured


## Execute Hyperparameter Search (3-6 hours for 75 trials)

In [ ]:
import time

print("="*80)
print("STARTING HYPERPARAMETER SEARCH")
print(f"Training set: {len(X_train)}, Validation set: {len(X_test)}")
print("Max epochs per trial: 20")
print("Estimated time: 3-6 hours")
print("="*80)

search_start = time.time()

tuner.search(X_train, y_train, epochs=20, validation_data=(X_test, y_test), callbacks=tuner_callbacks, batch_size=256, verbose=1)

search_time = time.time() - search_start
print(f"\nSearch complete: {search_time/3600:.2f} hours")

Trial 36 Complete [00h 03m 39s]
val_accuracy: 0.9111999869346619

Best val_accuracy So Far: 0.9132999777793884
Total elapsed time: 11h 36m 58s

Search: Running Trial #37

Value             |Best Value So Far |Hyperparameter
2                 |1                 |num_lstm_layers
50                |256               |lstm_units
0                 |0.45              |dropout
0.00020064        |0.001             |learning_rate

Epoch 1/20


/mnt/c/Users/minda/PycharmProjects/ImdbSentimentAnalysis/venv/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


157/157 ━━━━━━━━━━━━━━━━━━━━ 13s 75ms/step - accuracy: 0.6757 - loss: 0.5844 - val_accuracy: 0.7785 - val_loss: 0.4675
Epoch 2/20
157/157 ━━━━━━━━━━━━━━━━━━━━ 11s 71ms/step - accuracy: 0.8180 - loss: 0.4102 - val_accuracy: 0.8351 - val_loss: 0.3777
Epoch 3/20
157/157 ━━━━━━━━━━━━━━━━━━━━ 11s 70ms/step - accuracy: 0.8433 - loss: 0.3626 - val_accuracy: 0.8455 - val_loss: 0.3531
Epoch 4/20
149/157 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.8595 - loss: 0.3339

## Best Hyperparameters

In [ ]:
print("="*80)
print("BEST HYPERPARAMETERS:")
print("="*80)
best_hps = tuner.get_best_hyperparameters(1)[0]
print(f"LSTM Layers: {best_hps.get('num_lstm_layers')}")
print(f"LSTM Units: {best_hps.get('lstm_units')}")
print(f"Dropout: {best_hps.get('dropout'):.3f}")
print(f"Learning Rate: {best_hps.get('learning_rate'):.6f}")
print("\nComparison with original (2 layers, 100 units, 0.12 dropout, 0.00005 LR)")

## Top 5 Configurations

In [ ]:
print("TOP 5 CONFIGURATIONS:")
for i, trial in enumerate(tuner.oracle.get_best_trials(5), 1):
    print(f"\n#{i}: Accuracy={trial.score:.4f}")
    print(f"  Layers={trial.hyperparameters.get('num_lstm_layers')}, Units={trial.hyperparameters.get('lstm_units')}, Dropout={trial.hyperparameters.get('dropout'):.3f}, LR={trial.hyperparameters.get('learning_rate'):.6f}")

## Results Visualization

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

trial_data = []
for trial in tuner.oracle.trials.values():
    if trial.score is not None:
        trial_data.append({
            "trial_id": trial.trial_id,
            "val_accuracy": trial.score,
            "num_layers": trial.hyperparameters.get("num_lstm_layers"),
            "lstm_units": trial.hyperparameters.get("lstm_units"),
            "dropout": trial.hyperparameters.get("dropout"),
            "learning_rate": trial.hyperparameters.get("learning_rate")
        })

df = pd.DataFrame(trial_data)

fig, axes = plt.subplots(2, 3, figsize=(18, 10))
fig.suptitle("Hyperparameter Tuning Results", fontsize=16)

axes[0,0].plot(df["trial_id"], df["val_accuracy"], "o-")
axes[0,0].axhline(0.8885, color="red", linestyle="--", label="Original")
axes[0,0].set_title("Accuracy Progress")
axes[0,0].legend()

axes[0,1].scatter(df["lstm_units"], df["val_accuracy"], c=df["num_layers"], cmap="viridis", s=100, alpha=0.6)
axes[0,1].set_title("LSTM Units vs Accuracy")

axes[0,2].scatter(df["dropout"], df["val_accuracy"], c=df["lstm_units"], cmap="plasma", s=100, alpha=0.6)
axes[0,2].set_title("Dropout vs Accuracy")

axes[1,0].scatter(df["learning_rate"], df["val_accuracy"], s=100, alpha=0.6)
axes[1,0].set_xscale("log")
axes[1,0].set_title("Learning Rate vs Accuracy")

df.groupby("num_layers")["val_accuracy"].agg(["mean", "max"]).plot(kind="bar", ax=axes[1,1])
axes[1,1].set_title("Performance by Layers")

axes[1,2].text(0.5, 0.5, f"Best: {df['val_accuracy'].max():.4f}\nMean: {df['val_accuracy'].mean():.4f}\nStd: {df['val_accuracy'].std():.4f}", ha="center", va="center", fontsize=14)
axes[1,2].set_title("Summary Stats")
axes[1,2].axis("off")

plt.tight_layout()
plt.show()

print(f"Improvement over original: {(df['val_accuracy'].max() - 0.8885)*100:.2f}%")

## Train Final Model with Best Hyperparameters

In [ ]:
print("Training final model with best hyperparameters...")
best_model = tuner.hypermodel.build(best_hps)

final_early_stopping = EarlyStopping(monitor="val_accuracy", patience=5, restore_best_weights=True, mode="max", verbose=1)

final_history = best_model.fit(X_train, y_train, batch_size=256, epochs=50, validation_data=(X_test, y_test), callbacks=[final_early_stopping], verbose=1)

final_score, final_acc = best_model.evaluate(X_test, y_test, batch_size=256)
print(f"\nFinal Test Accuracy: {final_acc:.4f}")
print(f"Final Test Loss: {final_score:.4f}")
print(f"Improvement: {(final_acc - 0.8885)*100:.2f}%")

## Save Results

In [ ]:
import json
import os
from datetime import datetime

os.makedirs("models", exist_ok=True)

results = {
    "date": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "total_trials": len(df),
    "best_hyperparameters": {
        "num_lstm_layers": int(best_hps.get("num_lstm_layers")),
        "lstm_units": int(best_hps.get("lstm_units")),
        "dropout": float(best_hps.get("dropout")),
        "learning_rate": float(best_hps.get("learning_rate"))
    },
    "final_accuracy": float(final_acc),
    "final_loss": float(final_score),
    "improvement": float((final_acc - 0.8885) * 100)
}

with open("tuning_results_summary.json", "w") as f:
    json.dump(results, f, indent=2)

best_model.save("models/lstm_tuned_best.keras")
df.to_csv("tuning_history.csv", index=False)

print("Saved: tuning_results_summary.json")
print("Saved: models/lstm_tuned_best.keras")
print("Saved: tuning_history.csv")

## Test Predictions

In [ ]:
texts = ["This was the best movie I have ever seen!", "I really hated this film. It was slow and boring."]

def predict(text, model):
    seq = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(seq, maxlen=MAX_LEN)
    score = model.predict(padded, verbose=0)[0][0]
    return score, "Positive" if score > 0.5 else "Negative"

print("\nTuned Model Predictions:")
for text in texts:
    score, label = predict(text, best_model)
    print(f"{text[:50]}... -> {score:.4f} ({label})")